In [1]:
import numpy as np
import math

def rotation_matrix_x(angle_degrees):
    """ Erstellt eine Rotationsmatrix um die X-Achse (Pitch). """
    angle_radians = math.radians(angle_degrees)
    c = math.cos(angle_radians)
    s = math.sin(angle_radians)
    return np.array([
        [1, 0, 0, 0],
        [0, c, -s, 0],
        [0, s, c, 0],
        [0, 0, 0, 1]
    ])

def rotation_matrix_y(angle_degrees):
    """ Erstellt eine Rotationsmatrix um die Y-Achse (Yaw). """
    angle_radians = math.radians(angle_degrees)
    c = math.cos(angle_radians)
    s = math.sin(angle_radians)
    return np.array([
        [c, 0, s, 0],
        [0, 1, 0, 0],
        [-s, 0, c, 0],
        [0, 0, 0, 1]
    ])

def rotation_matrix_z(angle_degrees):
    """ Erstellt eine Rotationsmatrix um die Z-Achse (Roll). """
    angle_radians = math.radians(angle_degrees)
    c = math.cos(angle_radians)
    s = math.sin(angle_radians)
    return np.array([
        [c, -s, 0, 0],
        [s, c, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

np.set_printoptions(suppress=True, precision=3)

print("--- Szenario 1: Normale Rotation (kein Gimbal Lock) ---")

yaw_angle = 30  # Gieren
pitch_angle = 45 # Nicken
roll_angle = 60  # Rollen

# Erstellen der einzelnen Matrizen
R_yaw = rotation_matrix_y(yaw_angle)
R_pitch = rotation_matrix_x(pitch_angle)
R_roll = rotation_matrix_z(roll_angle)

# Kombinieren der Matrizen durch Multiplikation
# Wir wenden zuerst Roll an, dann Pitch, dann Yaw (also umgekehrt aufmultiplizieren)
combined_matrix_normal = R_yaw @ R_pitch @ R_roll

print(f"Rotation: Yaw={yaw_angle}°, Pitch={pitch_angle}°, Roll={roll_angle}°")
print("Kombinierte Matrix:")
print(combined_matrix_normal)
print("\nAnalyse: Die resultierende Matrix ist komplex. Jede der drei Rotationen hat einen einzigartigen Beitrag geleistet.")
print("-" * 50)


print("\n--- Szenario 2: Gimbal Lock ---")
# Pithc 90 Grad damit Lock passiert
pitch_angle_lock = 90
# Wir lassen Yaw und Roll variabel, um zu sehen, wie sie interagieren.
yaw_angle_lock = 30
roll_angle_lock = 60

print(f"Rotation: Yaw={yaw_angle_lock}°, Pitch={pitch_angle_lock}° (LOCK!), Roll={roll_angle_lock}°")

# Erstellen der Matrizen
R_yaw_lock = rotation_matrix_y(yaw_angle_lock)
R_pitch_lock = rotation_matrix_x(pitch_angle_lock) # Der Auslöser
R_roll_lock = rotation_matrix_z(roll_angle_lock)

# Kombinieren der Matrizen
combined_matrix_lock = R_yaw_lock @ R_pitch_lock @ R_roll_lock

print("\nMatrix für Pitch (Nicken) um 90°:")
print(R_pitch_lock)
print("\nKombinierte Matrix im Gimbal Lock:")
print(combined_matrix_lock)

print("\n--- Mathematische Analyse des Gimbal Lock ---")
print("Wenn Pitch = 90°, vereinfacht sich die Pitch-Matrix zu:")
print("cos(90) = 0, sin(90) = 1")
print("[[1, 0,  0, 0],")
print(" [0, 0, -1, 0],")
print(" [0, 1,  0, 0],")
print(" [0, 0,  0, 1]]")

print("\nWenn wir nun die Matrizen symbolisch multiplizieren (Yaw * Pitch(90) * Roll),")
print("stellen wir fest, dass die Terme für Yaw und Roll miteinander verschmelzen:")

# Symbolische Darstellung der resultierenden Matrixelemente
alpha_plus_gamma_rad = math.radians(yaw_angle_lock + roll_angle_lock)
c_a_g = math.cos(alpha_plus_gamma_rad)
s_a_g = math.sin(alpha_plus_gamma_rad)

# Manuelle Berechnung der Gimbal-Lock-Matrix zur Veranschaulichung
manual_lock_matrix_corner = np.array([
    [c_a_g, -s_a_g],
    [s_a_g, c_a_g]
])

print(f"\nDie obere linke Ecke der kombinierten Matrix hängt nur von der SUMME von Yaw und Roll ab: Yaw+Roll = {yaw_angle_lock+roll_angle_lock}°")
print(manual_lock_matrix_corner)
print("\nVergleichen  mit der oberen linken Ecke der berechneten 'combined_matrix_lock'.")
print(combined_matrix_lock[:2, :2])

--- Szenario 1: Normale Rotation (kein Gimbal Lock) ---
Rotation: Yaw=30°, Pitch=45°, Roll=60°
Kombinierte Matrix:
[[ 0.739 -0.573  0.354  0.   ]
 [ 0.612  0.354 -0.707  0.   ]
 [ 0.28   0.739  0.612  0.   ]
 [ 0.     0.     0.     1.   ]]

Analyse: Die resultierende Matrix ist komplex. Jede der drei Rotationen hat einen einzigartigen Beitrag geleistet.
--------------------------------------------------

--- Szenario 2: Gimbal Lock ---
Rotation: Yaw=30°, Pitch=90° (LOCK!), Roll=60°

Matrix für Pitch (Nicken) um 90°:
[[ 1.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]

Kombinierte Matrix im Gimbal Lock:
[[ 0.866 -0.5    0.     0.   ]
 [ 0.     0.    -1.     0.   ]
 [ 0.5    0.866  0.     0.   ]
 [ 0.     0.     0.     1.   ]]

--- Mathematische Analyse des Gimbal Lock ---
Wenn Pitch = 90°, vereinfacht sich die Pitch-Matrix zu:
cos(90) = 0, sin(90) = 1
[[1, 0,  0, 0],
 [0, 0, -1, 0],
 [0, 1,  0, 0],
 [0, 0,  0, 1]]

Wenn wir nun die Matrizen symbolisch multiplizi